<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-and-Load-Data" data-toc-modified-id="Import-and-Load-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import and Load Data</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocessing</a></span></li><li><span><a href="#Trying-Out-Models" data-toc-modified-id="Trying-Out-Models-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Trying Out Models</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#Support-Vector-Machine" data-toc-modified-id="Support-Vector-Machine-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Support Vector Machine</a></span></li><li><span><a href="#Decision-Trees-(Random-Forest,-Gradient-Boosting,-XGBoost)" data-toc-modified-id="Decision-Trees-(Random-Forest,-Gradient-Boosting,-XGBoost)-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Decision Trees (Random Forest, Gradient Boosting, XGBoost)</a></span></li><li><span><a href="#Other-Models-(e.g.-Bagging-Classifier)" data-toc-modified-id="Other-Models-(e.g.-Bagging-Classifier)-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Other Models (e.g. Bagging Classifier)</a></span></li></ul></li><li><span><a href="#Model-Evaluation" data-toc-modified-id="Model-Evaluation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Model Evaluation</a></span></li></ul></div>

## Import and Load Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [2]:
import pandas as pd
from google.colab import drive
import zipfile

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Extract the ZIP file containing the CSV
zip_ref = zipfile.ZipFile("/content/drive/Shareddrives/OMIS 116 Team Drive/Homework/loans.csv", 'r')
zip_ref.extractall("/content/dataset")
zip_ref.close()

# Assuming the CSV file is named loans.csv and is directly inside the zip without any folder structure
csv_file_path = "/content/dataset/loans.csv"

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)
#df = pd.read_csv('loans.csv')

BadZipFile: File is not a zip file

## Preprocessing

 - Handle missing values
 - Encode categorical variables, scale data (if you wish), feature selection, etc.
 - Split the dataset into features (X) and target variable (y)
 - Split into training and testing sets

In [ ]:
df.drop(['id','member_id','emp_title','title','zip_code','url'],axis=1,inplace=True)

In [ ]:
df['emp_length'] = df['emp_length'].str.extract('(\d+)').astype(float)

In [ ]:
df.columns

Index(['Unnamed: 0', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term',
       'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'desc', 'purpose', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_

In [ ]:
df_cleaned = df.copy()

In [ ]:
threshold = len(df) * 0.10 # 90% threshold

In [ ]:
df_cleaned = df.dropna(axis = 1, thresh = threshold) # dropped columns that have more than 80% missing values

In [ ]:
df.addr_state

0         AZ
1         GA
2         IL
3         CA
4         OR
          ..
887374    CA
887375    NJ
887376    TN
887377    MA
887378    FL
Name: addr_state, Length: 887379, dtype: object

In [ ]:
continuous_columns = ['loan_amnt', 'installment', 'funded_amnt','funded_amnt_inv', 'annual_inc', 'dti', \
                      'revol_bal', 'revol_util', 'total_rev_hi_lim','total_acc',\
                      'int_rate', 'pub_rec', 'delinq_2yrs','inq_last_6mths','open_acc','acc_now_delinq', 'emp_length'
                     ]
nominal_columns = ['home_ownership', 'pymnt_plan', 'term', 'application_type', 'initial_list_status', 'purpose',  'verification_status',\
                    'sub_grade', 'addr_state']
ordinal_columns = []
time_columns = ['earliest_cr_line']

In [ ]:
df.sub_grade.unique()

array(['B2', 'C4', 'C5', 'C1', 'B5', 'A4', 'E1', 'F2', 'C3', 'B1', 'D1',
       'A1', 'B3', 'B4', 'C2', 'D2', 'A3', 'A5', 'D5', 'A2', 'E4', 'D3',
       'D4', 'F3', 'E3', 'F4', 'F1', 'E5', 'G4', 'E2', 'G3', 'G2', 'G1',
       'F5', 'G5'], dtype=object)

In [ ]:
total_cols = nominal_columns + continuous_columns

In [ ]:
from sklearn.impute import SimpleImputer
preprocessor = ColumnTransformer(
    transformers=[
        ('nominal', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown = 'ignore'))
        ]), nominal_columns),
        ('continuous', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), continuous_columns),
    ],
    remainder='drop'
)

In [ ]:
# ,'Late (31-120 days)', 'Late (16-30 days)'

In [ ]:
df.loan_status.unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'],
      dtype=object)

In [ ]:
desired_statuses = ['Fully Paid', 'Default', 'Charged Off']

df_cleaned = df[df['loan_status'].isin(desired_statuses)]

In [ ]:
df_cleaned.loan_status.unique()

array(['Fully Paid', 'Charged Off', 'Default'], dtype=object)

In [ ]:
df_cleaned['binary_loan_status'] = df_cleaned['loan_status'].apply(lambda x: 1 if x in ['Fully Paid'] else 0)

/var/folders/vm/_8lh9zpx0vbc_x2bq621qrhc0000gn/T/ipykernel_90154/2618706337.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['binary_loan_status'] = df_cleaned['loan_status'].apply(lambda x: 1 if x in ['Fully Paid'] else 0)


In [ ]:
df_cleaned.drop(columns = 'loan_status', inplace = True)

/var/folders/vm/_8lh9zpx0vbc_x2bq621qrhc0000gn/T/ipykernel_90154/3704106794.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.drop(columns = 'loan_status', inplace = True)


In [ ]:
X = df_cleaned[total_cols]

In [ ]:
y = df_cleaned.binary_loan_status

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, shuffle = True, test_size=0.3)

In [ ]:
y_train.sum()

145429

In [ ]:
len(y_train)

177933

In [ ]:
training_loans_given_percent = y_train.sum() / len(y_train) * 100
print(f'Percentage of training loans given: {training_loans_given_percent:.2f}%')

Percentage of training loans given: 81.73%


In [ ]:
testing_loans_given_percent = y_test.sum() / len(y_test) * 100
print(f'Percentage of testing loans given: {testing_loans_given_percent:.2f}%')

Percentage of testing loans given: 81.69%


In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

pipeline.fit(X_train, y_train)
y_pred_proba = pipeline.predict_proba(X_test)[:,1]

auc_score = roc_auc_score(y_test, y_pred_proba)
print("AUC Score: ", auc_score)

AUC Score:  0.7047060488150438


In [ ]:
from sklearn.metrics import roc_curve, auc

# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=42, max_depth = 5)

# Setup the pipeline for preprocessing and model
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', dt_classifier)])

#pipeline.fit(X_train, y_train)
#y_pred_proba = pipeline.predict_proba(X_test)[:,1]

#auc_score = roc_auc_score(y_test, y_pred_proba)
#print("AUC Score: ", auc_score)

## Trying Out Models

Here, you want to try each type of machine learning model and perform the train-test-loop: identify the best hyperparameters for the model to perform well in training and validation. GridSearchCV is likely relevant.

### Logistic Regression